In [55]:
pip install redis pymupdf openai numpy


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [56]:
pip install openai --upgrade


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [57]:
import openai
from packaging import version

required_version = version.parse("1.1.1")
current_version = version.parse(openai.__version__)

if current_version < required_version:
    raise ValueError(f"Error: OpenAI version {openai.__version__}"
                     " is less than the required version 1.1.1")
else:
    print("OpenAI version is compatible.")

# -- Now we can get to it
from openai import OpenAI
client = OpenAI(api_key="sk-xxx")

OpenAI version is compatible.


In [58]:
from openai import OpenAI
client = OpenAI(api_key="")
def ask_chatgpt(messages):
  response = client.chat.completions.create(
    model="gpt-3.5-turbo", messages=messages
  )
  return response.choices[0].message.content


prompt_role = "You are an assistant for journalists. \
    Your task is to write articles, based on the FACTS that are given to you. \
    You should respect the instructions: the TONE, the LENGTH, and the STYLE"


from typing import List
def assist_journalist(
  facts: List[str], tone: str, length_words: int, style: str
):
  facts = ", ".join(facts)
  prompt_role = "You are an assistant for journalists"
  prompt = f"{prompt_role} \
    FACTS: {facts} \
    TONE: {tone} \
    LENGTH: {length_words} words \
    STYLE: {style}"
  return ask_chatgpt([{"role": "user", "content": prompt}])


print(
  assist_journalist(
    ["The sky is blue", "The grass is green"], "informal", 100, "blogpost"
  )
)

Hey there! As an assistant for journalists, I wanted to share some fun facts with you today. Did you know that the sky is blue and the grass is green? It may seem like common knowledge, but it's always good to refresh our minds on the basics. Next time you're out enjoying nature, take a moment to appreciate the beautiful colors that surround us. And if you're working on a story about the great outdoors, don't forget to mention the blue sky and green grass – it's the little details that make a big impact! Happy writing!


- ''.join(리스트)''.join(리스트)를 이용하면 매개변수로 들어온 ['a', 'b', 'c'] 이런 식의 리스트를 'abc'의 문자열로 합쳐서 반환해주는 함수인 것입니다.

In [59]:
print(
    assist_journalist(
        facts = [
            "A book on ChatGpt has been published last week",
            "The title is Developing Apps with GPT-4 and CHATGPT",
            "The publisher is O'Reilly.", ],
        tone = "excited",
        length_words = 50,
        style = "news flash",))

Exciting news for tech enthusiasts! A new book on ChatGpt has hit the shelves titled "Developing Apps with GPT-4 and CHATGPT." Published by O'Reilly, this book is a must-have for anyone looking to explore the potential of this cutting-edge technology. Grab your copy now and stay ahead of the curve!


In [60]:
with open("trans.txt", "r") as f:
    transcript = f.read()

response = client.chat.completions.create(
    model = "gpt-4-1106-preview",
    messages = [
        {"role":"system","content":"You are a helpful assistant."},
        {"role":"user","content":"Summarize the following text"},
        {"role":"assistant","content":"Yes."},
        {"role":"user","content":transcript}],)
print(response.choices[0].message.content)

Elon Musk, an influential entrepreneur known for popularizing electric vehicles and working on various projects like reusable rockets, neural technology, high-speed trains, and satellite systems aiding Ukraine, appeared as a guest on a show. During his interview, he discussed his busy schedule and multiple initiatives, including his extensive workdays.

Despite attacks from critics, Musk maintains a sense of humor and highlights the importance of humor and humility. The conversation touched on the significance of innovators in tech shaping the world, the role of technology in civilization, the value of free speech, and the danger of 'woke' culture suppressing freedom of expression and promoting anti-meritocracy.

Musk also shared his thoughts on education and the spread of misinformation, pointing out gaps in knowledge and awareness among the younger generation. Elon and the host discussed AI's potential threat to humanity and the importance of regulatory oversight to prevent companies

In [61]:
class DataService():
    def __init__(self):
        self.redis_client = redis.Redis(
            host= REDIS_HOST,
            port=REDIS_PORT,
            password = REDIS_PASSWORD)

In [62]:
def pdf_to_embeddings(self, pdf_path: str, chunk_length: int = 1000):
# PDF 파일에서 데이터를 읽고 여러 부분으로 분할
    reader = PdfReader(pdf_path)
    chunks = []
    for page in reader.pages:
        text_page = page.extract_text()
        chunks.extend([text_page[i:i+chunk_length]
            for i in range(0, len(text_page), chunk_length)])
# 임베딩 생성
    response = client.embeddings.create(model='text-embedding-ada-002',
    input=chunks)
    return [{'id': value['index'],
    'vector':value['embedding'],
    'text':chunks[value['index']]} for value in response['choices']]


In [63]:
def load_data_to_redis(self, embeddings):
  for embedding in embeddings:
      key = f"{PREFIX}:{str(embedding['id'])}"
      embedding["vector"] = np.array(
          embedding["vector"], dtype=np.float32).tobytes()
      self.redis_client.hset(key, mapping=embedding)

In [64]:
def search_redis(self,user_query: str):
# 사용자 쿼리에서 임베딩 벡터 생성
    embedded_query = client.embeddings.create(
      input=user_query,
      model="text-embedding-ada-002")["data"][0]['embedding']

# 벡터 검색 수행
    results = self.redis_client.ft(index_name).search(query, params_dict)
    return [doc['text'] for doc in results.docs]

In [65]:
class IntentService():
  def __init__(self):
    pass
  def get_intent(self, user_question: str):
      # 오픈AI 채팅 완성 엔드포인트 호출
      response = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[
              {"role": "user",
               "content": f"""Extract the keywords from the following
                question: {user_question}."""}
          ]
      )
      # 응답 추출
      return (response.choices[0].message.content)


In [66]:
class ResponseService():
  def __init__(self):
    pass
  def generate_response(self, facts, user_question):
      # 오픈AI 채팅 완성 엔드포인트 호출
      response = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[
                {"role": "user",
                 "content": f"""Based on the FACTS, answer the QUESTION.
                  QUESTION: {user_question}. FACTS: {facts}"""}
          ]
      )
      # 응답 추출
      return (response.choices[0].message.content)



In [69]:
def run(question: str, file: str='ExplorersGuide.pdf'):
    data_service = DataService()
    data = data_service.pdf_to_embeddings(file)
    data_service.load_data_to_redis(data)
